## 1. 필수 라이브러리 임포트 및 환경 설정
- .env에 openai api key 저장: sk-.....

In [15]:
import os
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
# from langchain.chains import LLMChain -> legacy
from langchain_core.runnables import RunnableParallel, RunnableMap, RunnablePassthrough
from dotenv import load_dotenv
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response

# env 파일 읽기
load_dotenv()

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI_report", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
AI_report


## 2. Sample Data 생성
- postgre와 연결 필요

In [16]:
# 날짜 생성 (2021-01 ~ 2024-09)
dates = pd.date_range(start="2021-01-01", end="2024-09-01", freq='MS').strftime("%Y-%m").tolist()

# 데이터 생성
data = {
    '날짜': dates,
    'ROA': np.random.uniform(3, 8, len(dates)),  # 3~8% 사이의 임의의 ROA 값
    '노동생산성': np.random.uniform(1, 3, len(dates)),  # 1~3 사이의 임의의 노동생산성 값
    '당좌비율': np.random.uniform(1, 1.5, len(dates)),  # 1~1.5 사이의 임의의 당좌비율 값
    '매출증가율': np.random.uniform(5, 15, len(dates)),  # 5~15% 사이의 매출증가율 값
    '부채비율': np.random.uniform(40, 70, len(dates)),  # 40~70% 사이의 부채비율 값
    '순이익증가율': np.random.uniform(5, 20, len(dates)),  # 5~20% 사이의 순이익증가율 값
    '순이익률': np.random.uniform(4, 10, len(dates)),  # 4~10% 사이의 순이익률 값
    '영업이익률': np.random.uniform(5, 12, len(dates)),  # 5~12% 사이의 영업이익률 값
    '유동비율': np.random.uniform(1.3, 2, len(dates)),  # 1.3~2 사이의 유동비율 값
    '이자보상배율': np.random.uniform(2, 5, len(dates)),  # 2~5 사이의 이자보상배율 값
    '자기자본비율': np.random.uniform(30, 60, len(dates)),  # 30~60% 사이의 자기자본비율 값
    '총자산증가율': np.random.uniform(3, 10, len(dates)),  # 3~10% 사이의 총자산증가율 값
    '자산회전율': np.random.uniform(1, 2, len(dates)),  # 1~2 사이의 자산회전율 값
    '재고자산회전율': np.random.uniform(2, 4, len(dates)),  # 2~4 사이의 재고자산회전율 값
    '현금흐름비율': np.random.uniform(1, 1.5, len(dates))  # 1~1.5 사이의 현금흐름비율 값
}

# DataFrame 생성
df_financial = pd.DataFrame(data)

# 샘플 출력
print(df_financial)

         날짜       ROA     노동생산성      당좌비율      매출증가율       부채비율     순이익증가율  \
0   2021-01  5.502876  2.544676  1.188908   9.803900  51.893094   5.650064   
1   2021-02  6.021979  2.946321  1.482327   9.782756  65.416244   8.300307   
2   2021-03  7.993784  1.150558  1.003166  12.167989  47.220527  15.302069   
3   2021-04  7.399974  1.600339  1.197240   8.273596  65.273102  14.773713   
4   2021-05  4.801456  1.943163  1.122709   6.950597  49.229099  17.291457   
5   2021-06  7.648269  1.363561  1.327039  10.888743  65.499933  19.375470   
6   2021-07  3.157839  2.786276  1.221361   6.091245  44.598627   8.217840   
7   2021-08  5.877362  2.957669  1.263533  14.429723  61.138580  18.184838   
8   2021-09  4.584442  2.648429  1.424143  12.895772  43.733162  16.242404   
9   2021-10  4.784553  2.129715  1.210896   8.505993  45.497751  18.743529   
10  2021-11  6.473248  1.354596  1.012760  14.899334  57.846123  18.120036   
11  2021-12  4.714708  2.127659  1.294779  11.447087  63.121127 

- LLM에 들어갈 포맷으로 변환

In [17]:
# preprocessing.py에 저장한 전처리 함수를 import
from preprocessing import extract_latest_month_and_previous_year, calculate_three_year_average, preprocess_financial_data, assign_chain_data
import pprint # 출력 예시

# JSON으로 변환하기 위해 DataFrame 처리
processed_financial_data = preprocess_financial_data(df_financial)

pprint.pprint(processed_financial_data)

{'latest_year_month': '2024-09',
 'previous_year_month': '2023-09',
 'processed_data': {'ROA': {'3년 평균 대비 변화율 (%)': -24.74,
                            '전년 동기 대비 변화율 (%)': -10.82,
                            '전년 동기 데이터': 3.65,
                            '직전년도 3년 평균': 4.33,
                            '최근 12개월 데이터': {'2023-09': 3.65,
                                            '2023-10': 7.1,
                                            '2023-11': 7.28,
                                            '2023-12': 7.69,
                                            '2024-01': 4.26,
                                            '2024-02': 3.68,
                                            '2024-03': 5.63,
                                            '2024-04': 3.27,
                                            '2024-05': 5.34,
                                            '2024-06': 4.08,
                                            '2024-07': 6.88,
                                            '2024-08': 

- 각 Chain에 입력된 데이터 분할

In [18]:
# Preprocessed 데이터를 각 체인에 맞게 세팅
input_data = assign_chain_data(processed_financial_data)

pprint.pprint(input_data)

{'growth_data': {'매출증가율': {'3년 평균 대비 변화율 (%)': -45.96,
                           '전년 동기 대비 변화율 (%)': -48.53,
                           '전년 동기 데이터': 13.55,
                           '직전년도 3년 평균': 12.9,
                           '최근 12개월 데이터': {'2023-09': 13.55,
                                           '2023-10': 13.66,
                                           '2023-11': 7.05,
                                           '2023-12': 13.77,
                                           '2024-01': 14.02,
                                           '2024-02': 6.68,
                                           '2024-03': 13.33,
                                           '2024-04': 7.83,
                                           '2024-05': 7.63,
                                           '2024-06': 9.05,
                                           '2024-07': 12.01,
                                           '2024-08': 14.5,
                                           '2024-09': 6.97},
         

## 2. 프롬프트 파일 읽기 및 템플릿 생성 함수 정의

In [19]:

# 현재 파일이 실행 중인 절대 경로
# CURRENT_FILE_PATH = os.path.abspath(__file__) # py용
CURRENT_DIR = os.getcwd()
PROMPT_PATH = os.path.join(CURRENT_DIR, "prompts")

# prompts 폴더에서 prompt txt 파일 읽기
def load_prompt(file_name):
    """
    주어진 파일 이름의 프롬프트 템플릿을 읽어 반환합니다.
    """
    file_path = os.path.join(PROMPT_PATH, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    
# 모든 프롬프트 템플릿 로드    
system_template = load_prompt("system_template.txt")
growth_template = load_prompt("growth_template.txt")
profitability_template = load_prompt("profitability_template.txt")
stability_template = load_prompt("stability_template.txt")
productivity_template = load_prompt("productivity_template.txt")
liquidity_template = load_prompt("liquidity_template.txt")
solution_template = load_prompt("solution_template.txt")

print(system_template)
print('\n\n')
print(growth_template)

<Persona>
  당신은 중소기업 CEO를 위한 경영진단 보고서를 작성하는 AI입니다.
</Persona>

<Instruction>
  1. 모든 분석은 최신 월({latest_year_month})을 기준으로 전년 동기({previous_year_month})와 직전 12개월 데이터를 포함하여 수행하세요.
  2. 분석은 최대한 객관적인 사실에 기반하며, 변화율과 비교 지표를 명확히 제시하세요.
  3. 각각의 분석 체인은 간결하게 3줄 이내로 요약하세요.
</Instruction>





다음은 최신 월({latest_year_month}) 기준으로 주어진 성장성 관련 데이터입니다:
{growth_data}

<INSTRUCTION>
  1. 매출 증가율, 총자산 증가율, 순이익 증가율에 대해 **전년 동기 대비**, **3년 평균 대비**, 그리고 **최근 12개월 추이**를 순차적으로 분석하세요.
  2. 각 지표의 분석 결과를 종합한 후, **객관적인 수치와 변화율**을 포함하여 **최종 요약된 결과를 3줄 이내로** 간단히 작성하세요.
</INSTRUCTION>



## 3. PromptTemplate 객체 생성

In [20]:

# # PromptTemplate 객체 생성 (시스템 프롬프트와 각 분석 프롬프트 결합)
# def create_prompt(system, specific):
#     return PromptTemplate.from_template(system + specific)

# growth_prompt = create_prompt(system_template, growth_template)
# profitability_prompt = create_prompt(system_template, profitability_template)
# stability_prompt = create_prompt(system_template, stability_template)
# productivity_prompt = create_prompt(system_template, productivity_template)
# liquidity_prompt = create_prompt(system_template, liquidity_template)
# solution_prompt = create_prompt(system_template, solution_template)

# Create prompts for individual analysis chains without system_template
growth_prompt = PromptTemplate.from_template(growth_template)
profitability_prompt = PromptTemplate.from_template(system_template + profitability_template)
stability_prompt = PromptTemplate.from_template(stability_template)
productivity_prompt = PromptTemplate.from_template(productivity_template)
liquidity_prompt = PromptTemplate.from_template(liquidity_template)
solution_prompt = PromptTemplate.from_template(solution_template)

print(growth_prompt)
print('\n\n')
print(productivity_prompt)
print("\n\n")
print(solution_prompt)

input_variables=['growth_data', 'latest_year_month'] template='다음은 최신 월({latest_year_month}) 기준으로 주어진 성장성 관련 데이터입니다:\n{growth_data}\n\n<INSTRUCTION>\n  1. 매출 증가율, 총자산 증가율, 순이익 증가율에 대해 **전년 동기 대비**, **3년 평균 대비**, 그리고 **최근 12개월 추이**를 순차적으로 분석하세요.\n  2. 각 지표의 분석 결과를 종합한 후, **객관적인 수치와 변화율**을 포함하여 **최종 요약된 결과를 3줄 이내로** 간단히 작성하세요.\n</INSTRUCTION>\n'



input_variables=['latest_year_month', 'previous_year_month', 'productivity_data'] template='다음은 최신 월({latest_year_month}) 기준으로 주어진 생산성 관련 노동생산성, 자산회전율, 재고자산회전율 데이터입니다:\n{productivity_data}\n\t1. **최신 월**({latest_year_month}) 기준으로 **전년 동기({previous_year_month})** 대비 변화율을 해석하세요.\n\t2. **최근 12개월** 동안의 월별 데이터를 분석하여 **계절적 패턴**과 **월별 추이**를 설명하세요.'



input_variables=['growth_analysis', 'liquidity_analysis', 'productivity_analysis', 'profitability_analysis', 'stability_analysis'] template='<PERSONA>\n  당신은 중소기업 CEO를 위한 경영진단 보고서를 작성하는 AI입니다.\n</PERSONA>\n\n다음은 기업의 최신 재무 성과 분석 결과입니다:\n\t1. 성장성 분석: {growth_analysis}\n\t2. 수익성 분석: {profitability_analysis

## 4. LLM 초기화

In [21]:
# 공통 LLM 설정 (자유도 낮게 설정)
common_llm = ChatOpenAI(
    temperature=0.2, 
    model_name="gpt-4o-mini", 
    openai_api_key=openai_api_key
)

# 경영 솔루션 제안 LLM (자유도 높게 설정)
solution_llm = ChatOpenAI(
    temperature=1.2, 
    model_name="gpt-4o",
    openai_api_key=openai_api_key
)  # 창의적인 접근 허용

solution_llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002898A662610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002898A6EA250>, root_client=<openai.OpenAI object at 0x000002898A661ED0>, root_async_client=<openai.AsyncOpenAI object at 0x000002898A6FA6D0>, model_name='gpt-4o', temperature=1.2, openai_api_key=SecretStr('**********'), openai_proxy='')

# 5. Chain 정의

In [22]:
# 각 분석 체인을 PromptTemplate과 LLM을 연결하여 정의
growth_chain = growth_prompt | common_llm
profitability_chain = profitability_prompt | common_llm
stability_chain = stability_prompt | common_llm
productivity_chain = productivity_prompt | common_llm
liquidity_chain = liquidity_prompt | common_llm

# 솔루션 체인 정의
solution_chain = solution_prompt | solution_llm

solution_chain

PromptTemplate(input_variables=['growth_analysis', 'liquidity_analysis', 'productivity_analysis', 'profitability_analysis', 'stability_analysis'], template='<PERSONA>\n  당신은 중소기업 CEO를 위한 경영진단 보고서를 작성하는 AI입니다.\n</PERSONA>\n\n다음은 기업의 최신 재무 성과 분석 결과입니다:\n\t1. 성장성 분석: {growth_analysis}\n\t2. 수익성 분석: {profitability_analysis}\n\t3. 재무 안정성 분석: {stability_analysis}\n\t4. 생산성 분석: {productivity_analysis}\n\t5. 유동성 분석: {liquidity_analysis}\n\n<INSTRUCTION>\n\t- 이 결과를 바탕으로 기업의 전반적인 재무 상태를 평가하고, 향후 성장을 위한 경영 전략을 제안하세요.\n\t- 평가 및 경영 전략은 간결하게 3줄 이내로 작성합니다.\n</INSTRUCTION>')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002898A662610>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002898A6EA250>, root_client=<openai.OpenAI object at 0x000002898A661ED0>, root_async_client=<openai.AsyncOpenAI object at 0x000002898A6FA6D0>, model_name='gpt-4o', temperature=1.2, openai_api_key=SecretStr('**********'), openai_proxy='')

In [9]:
input_data["growth_data"]

{'매출증가율': {'최신 데이터': 12.84,
  '전년 동기 데이터': 8.17,
  '전년 동기 대비 변화율 (%)': 57.21,
  '직전년도 3년 평균': 8.86,
  '3년 평균 대비 변화율 (%)': 44.9,
  '최근 12개월 데이터': {'2023-09': 8.17,
   '2023-10': 10.39,
   '2023-11': 6.33,
   '2023-12': 5.14,
   '2024-01': 6.0,
   '2024-02': 13.21,
   '2024-03': 8.91,
   '2024-04': 6.83,
   '2024-05': 6.72,
   '2024-06': 8.78,
   '2024-07': 9.12,
   '2024-08': 11.22,
   '2024-09': 12.84}},
 '총자산증가율': {'최신 데이터': 6.17,
  '전년 동기 데이터': 9.78,
  '전년 동기 대비 변화율 (%)': -36.92,
  '직전년도 3년 평균': 7.26,
  '3년 평균 대비 변화율 (%)': -15.07,
  '최근 12개월 데이터': {'2023-09': 9.78,
   '2023-10': 7.86,
   '2023-11': 5.14,
   '2023-12': 3.43,
   '2024-01': 5.76,
   '2024-02': 5.07,
   '2024-03': 6.06,
   '2024-04': 6.82,
   '2024-05': 9.52,
   '2024-06': 5.46,
   '2024-07': 3.22,
   '2024-08': 4.07,
   '2024-09': 6.17}},
 '순이익증가율': {'최신 데이터': 12.09,
  '전년 동기 데이터': 9.83,
  '전년 동기 대비 변화율 (%)': 22.93,
  '직전년도 3년 평균': 14.05,
  '3년 평균 대비 변화율 (%)': -13.99,
  '최근 12개월 데이터': {'2023-09': 9.83,
   '2023-10': 14.

In [23]:
# growth_chain에 필요한 데이터만 전달
growth_input = {
    "growth_data": input_data["growth_data"],
    "latest_year_month": input_data["latest_year_month"],
    "previous_year_month": input_data["previous_year_month"]
}

answer = growth_chain.stream(growth_input)

stream_response(answer)

1. **매출 증가율 분석**:
   - **전년 동기 대비**: 최신 매출 증가율은 6.97%로, 전년 동기 13.55%에서 -48.53% 감소했습니다.
   - **3년 평균 대비**: 3년 평균 12.9%에 비해 -45.96% 감소하여 성장세가 크게 둔화되었습니다.
   - **최근 12개월 추이**: 최근 12개월 동안 매출 증가율은 변동성이 있으며, 2024년 9월에는 6.97%로 가장 낮은 수치를 기록했습니다.

2. **총자산 증가율 분석**:
   - **전년 동기 대비**: 최신 총자산 증가율은 3.89%로, 전년 동기 8.4%에서 -53.67% 감소했습니다.
   - **3년 평균 대비**: 3년 평균 7.82%에 비해 -50.24% 감소하여 자산 성장도 둔화되었습니다.
   - **최근 12개월 추이**: 최근 12개월 동안 총자산 증가율은 3.89%로, 지속적으로 낮은 수준을 유지하고 있습니다.

3. **순이익 증가율 분석**:
   - **전년 동기 대비**: 최신 순이익 증가율은 13.56%로, 전년 동기 17.21%에서 -21.23% 감소했습니다.
   - **3년 평균 대비**: 3년 평균 14.57%에 비해 -6.94% 감소하여 순이익 성장세도 약화되었습니다.
   - **최근 12개월 추이**: 최근 12개월 동안 순이익 증가율은 13.56%로, 변동성이 있지만 여전히 양호한 수준을 유지하고 있습니다.

**최종 요약**: 매출 증가율은 6.97%로 전년 동기 대비 -48.53% 감소, 총자산 증가율은 3.89%로 -53.67% 감소, 순이익 증가율은 13.56%로 -21.23% 감소하여 전반적으로 성장세가 둔화되고 있습니다.

In [24]:
profitability_data = {
    "profitability_data" : input_data['profitability_data'],
    "latest_year_month": input_data["latest_year_month"],
    "previous_year_month": input_data["previous_year_month"]      
}

answer = profitability_chain.stream(profitability_data)

stream_response(answer)

### 1. 최신 월(2024-09) 기준으로 전년 동기(2023-09) 대비 변화율 해석

- **영업이익률**: 11.84%로 전년 동기 대비 3.83% 증가하였으며, 이는 직전 3년 평균(8.56%) 대비 38.34% 상승한 수치입니다. 이는 기업의 운영 효율성이 개선되었음을 나타냅니다.
  
- **순이익률**: 5.62%로 전년 동기 대비 21.22% 증가하였으나, 직전 3년 평균(6.85%) 대비 17.89% 감소하였습니다. 이는 단기적으로는 긍정적이나, 장기적으로는 수익성 개선이 필요함을 시사합니다.

- **ROA**: 3.26%로 전년 동기 대비 10.82% 감소하였으며, 직전 3년 평균(4.33%) 대비 24.74% 하락했습니다. 이는 자산 활용 효율성이 저하되었음을 의미합니다.

### 2. 최근 12개월 동안의 월별 데이터 분석

- **영업이익률**: 2023년 9월부터 2024년 8월까지 변동성이 있었으나, 2024년 9월에 11.84%로 회복되었습니다. 특히 2024년 3월(11.86%)과 2024년 9월(11.84%)에서 높은 수치를 기록하며, 전반적으로 안정적인 추세를 보였습니다.

- **순이익률**: 2023년 9월 4.64%에서 시작하여 2024년 11월에 9.57%로 급증한 후, 이후 다시 5%대에서 변동하였습니다. 이는 특정 월에 비정상적인 수익이 발생했음을 나타내며, 전반적으로는 5%대에서 안정세를 보이고 있습니다.

- **ROA**: 2023년 9월 3.65%에서 시작하여 2024년 1월 4.26%로 증가한 후, 2024년 9월에는 3.26%로 다시 감소했습니다. 이는 자산 활용의 일관성이 부족함을 나타내며, 특정 월에 비해 전반적으로 하락세를 보였습니다.

In [16]:
def create_analysis_parallel():
    return RunnableParallel({
        "growth_analysis": growth_chain,
        "profitability_analysis": profitability_chain,
        "stability_analysis": stability_chain,
        "productivity_analysis": productivity_chain,
        "liquidity_analysis": liquidity_chain,
        "latest_year_month": lambda x: x["latest_year_month"],
        "previous_year_month": lambda x: x["previous_year_month"]
    })
    
analysis_results_chain = create_analysis_parallel()

'''
chain_prompts = {
    "growth": growth_template,
    "profitability": profitability_template,
    "stability": stability_template,
    "productivity": productivity_template,
    "liquidity": liquidity_template
}

analysis_chains = {
    name: PromptTemplate.from_template(template) | create_llm_with_streaming(0.2, "gpt-4", openai_api_key)
    for name, template in chain_prompts.items()
}
'''

In [17]:
# 전체 체인 구성 수정
# overall_chain = (
#     RunnablePassthrough()  # 입력 데이터를 그대로 넘김
#     | create_analysis_parallel()  # 분석 체인 병렬 실행
#     | solution_chain  # 최종 솔루션 체인 실행
# )

overall_chain = (
    analysis_results_chain  # 분석 체인 병렬 실행
    | solution_chain  # 최종 솔루션 체인 실행
)

overall_chain

RunnablePassthrough()
| {
    growth_analysis: PromptTemplate(input_variables=['growth_data', 'latest_year_month', 'previous_year_month'], template='<Persona>\n  당신은 중소기업 CEO를 위한 경영진단 보고서를 작성하는 AI입니다.\n</Persona>\n\n<Instruction>\n  1. 모든 분석은 최신 월({latest_year_month})을 기준으로 전년 동기({previous_year_month})와 직전 12개월 데이터를 포함하여 수행하세요.\n  2. 분석은 최대한 객관적인 사실에 기반하며, 변화율과 비교 지표를 명확히 제시하세요.\n  3. 각각의 분석 체인은 사실 위주의 분석을 수행하고, 경영 전략 체인은 창의적인 제안을 제공하도록 해야 합니다.\n  4. 각각의 분석 체인은 간결하게 3줄 이내로 요약하세요.\n</Instruction>\n\n다음은 최신 월({latest_year_month}) 기준으로 주어진 성장성 관련 매출 성장률, 자산 성장률, 순이익 성장률 데이터입니다:\n{growth_data}\n  1. **최신 월**({latest_year_month}) 기준으로 **전년 동기({previous_year_month})** 대비 변화율을 해석하세요.\n  2. **최근 12개월** 동안의 월별 데이터를 분석하여 **계절적 패턴**과 **월별 추이**를 설명하세요.')
                     | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002B32AEDDDD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002B32C5D06D0>, root_client=<openai.OpenAI objec

In [19]:
# Execute the chain
result = overall_chain.invoke(input_data)

# 결과 출력
print(result.content)

content='### 재무 상태 평가\n\n1. **성장성**\n   - 매출 성장률이 전년 동기 대비 28.8% 증가(2024-09 기준)했으나, 직전 3년 평균보다는 낮음.\n   - 연말과 연초 매출이 고점, 4월과 5월의 매출이 저점.\n\n2. **수익성**\n   - 영업이익률은 38.35% 감소했으나, 순이익률은 1.21% 증가.\n   - ROA가 27.57% 감소하여, 자산 운영 효율성이 저하된 것으로 판단됨.\n\n3. **재무 안정성**\n   - 부채비율이 36.96% 감소, 이자보상배율 97.79% 증가, 자기자본비율 47.23% 증가로 재정 안정성 개선.\n\n4. **생산성**\n   - 노동생산성 84.7% 증가, 자산회전율 1.92% 감소, 재고자산회전율 5.72% 감소.\n\n5. **유동성**\n   - 유동비율이 6.35% 감소(유동성 약화), 현금흐름비율이 11.69% 증가, 당좌비율이 4.78% 증가.\n\n### 경영 전략 제안\n\n1. **비용 효율성**\n   - 비용 절감과 내부 프로세스 최적화를 통한 영업이익률 개선.\n   - IT 인프라 및 자동화 프로젝트 투자로 운영 효율성을 극대화.\n \n2. **자본 활용**\n   - 고수익 제품 라인 강화과 마케팅 투자 확대.\n   - 자산 재배치 및 비핵심 자산 매각을 통한 ROA 개선.\n\n3. **리스크 관리**\n   - 재무 데이터 변동성 줄이기위한 분기별 재무 성과 검토 체계 강화.\n   - 외환 리스크, 가격 리스크 등의 금융 리스크 헤징 전략 수립.\n\n4. **새로운 시장 기회**\n   - 신제품 출시 및 서비스를 위한 시장 조사 강화.\n   - 해외 시장 진출 기회 모색과 주요 이벤트 직전 프로모션 집중.\n\n종합적으로, 재무 상태는 전반적으로 개선되는 추세지만 특정 중요 지표들은 지속적인 관리와 개선이 필요합니다. 각 전략 목표를 달성하기 위해서는 부서별 솔루션을 세부적으로 수립하는 것이 중요합니다.' additional_kwar

In [27]:
# Split the content by sections, for example by double new lines or headers
sections = result.content.split("\n\n")

# Example: print the specific sections
for section in sections:
    print(section)

### 재무 상태 평가
1. **성장성**
   - 매출 성장률이 전년 동기 대비 28.8% 증가(2024-09 기준)했으나, 직전 3년 평균보다는 낮음.
   - 연말과 연초 매출이 고점, 4월과 5월의 매출이 저점.
2. **수익성**
   - 영업이익률은 38.35% 감소했으나, 순이익률은 1.21% 증가.
   - ROA가 27.57% 감소하여, 자산 운영 효율성이 저하된 것으로 판단됨.
3. **재무 안정성**
   - 부채비율이 36.96% 감소, 이자보상배율 97.79% 증가, 자기자본비율 47.23% 증가로 재정 안정성 개선.
4. **생산성**
   - 노동생산성 84.7% 증가, 자산회전율 1.92% 감소, 재고자산회전율 5.72% 감소.
5. **유동성**
   - 유동비율이 6.35% 감소(유동성 약화), 현금흐름비율이 11.69% 증가, 당좌비율이 4.78% 증가.
### 경영 전략 제안
1. **비용 효율성**
   - 비용 절감과 내부 프로세스 최적화를 통한 영업이익률 개선.
   - IT 인프라 및 자동화 프로젝트 투자로 운영 효율성을 극대화.
 
2. **자본 활용**
   - 고수익 제품 라인 강화과 마케팅 투자 확대.
   - 자산 재배치 및 비핵심 자산 매각을 통한 ROA 개선.
3. **리스크 관리**
   - 재무 데이터 변동성 줄이기위한 분기별 재무 성과 검토 체계 강화.
   - 외환 리스크, 가격 리스크 등의 금융 리스크 헤징 전략 수립.
4. **새로운 시장 기회**
   - 신제품 출시 및 서비스를 위한 시장 조사 강화.
   - 해외 시장 진출 기회 모색과 주요 이벤트 직전 프로모션 집중.
종합적으로, 재무 상태는 전반적으로 개선되는 추세지만 특정 중요 지표들은 지속적인 관리와 개선이 필요합니다. 각 전략 목표를 달성하기 위해서는 부서별 솔루션을 세부적으로 수립하는 것이 중요합니다.


In [30]:
# 개별 체인 결과 출력 추가
analysis_results = create_analysis_parallel().invoke(input_data)

# 개별 분석 결과 출력
print("Growth Analysis Result:\n", analysis_results["growth_analysis"].content)
print("Profitability Analysis Result:\n", analysis_results["profitability_analysis"].content)
print("Stability Analysis Result:\n", analysis_results["stability_analysis"].content)
print("Productivity Analysis Result:\n", analysis_results["productivity_analysis"].content)
print("Liquidity Analysis Result:\n", analysis_results["liquidity_analysis"].content)

성장성 분석 결과:
 content='### 분석 체인\n\n1. **최신 월(2024-09) 기준 전년 동기(2023-09) 대비 변화율 해석**\n   - 2024년 9월 매출 성장률은 6.72%로, 전년 동기인 2023년 9월의 5.22% 대비 28.8% 증가했습니다.\n   - 이는 전년 동기 대비 긍정적인 성장을 나타내며, 매출 증가가 지속되고 있음을 시사합니다.\n   - 그러나 3년 평균 대비 변화율은 -32.7%로, 장기적인 성장률은 감소 추세에 있습니다.\n\n2. **최근 12개월 동안의 월별 데이터 분석: 계절적 패턴과 월별 추이**\n   - 월별 매출 성장률은 2023년 11월과 2024년 1월, 2월에 급격히 상승했으며, 이는 연말 및 연초의 계절적 요인으로 보입니다.\n   - 2024년 4월과 5월에는 성장률이 상대적으로 낮아졌으나, 7월과 8월에 다시 상승하는 패턴을 보였습니다.\n   - 전반적으로 매출 성장률은 계절적 요인에 따라 변동이 크며, 특정 월에 급격한 상승과 하락이 반복되고 있습니다.\n\n### 경영 전략 체인\n\n1. **계절적 패턴을 활용한 마케팅 전략**\n   - 연말과 연초에 매출이 급증하는 경향을 활용하여, 이 시기에 맞춘 집중적인 마케팅 캠페인을 계획하세요.\n   - 할인 행사, 프로모션, 신규 제품 출시 등을 통해 매출 상승을 극대화할 수 있습니다.\n\n2. **저성장 월에 대한 대응 전략**\n   - 매출 성장률이 낮은 4월과 5월에는 고객 유입을 늘리기 위한 특별 이벤트나 프로모션을 진행하세요.\n   - 고객 충성도를 높이기 위한 멤버십 프로그램이나 로열티 프로그램을 강화하여, 비수기에도 안정적인 매출을 유지할 수 있습니다.\n\n3. **장기적인 성장률 개선 방안**\n   - 3년 평균 대비 성장률이 감소하고 있으므로, 장기적인 성장 전략을 재검토하세요.\n   - 신시장 개척, 제품 다각화, 기술 혁신 등을 통해 지속 가능한 성장을 도모할 필요가 있습니다.' additional_kw

In [31]:
# 각 분석 체인을 개별적으로 실행
growth_result = growth_chain.invoke(input_data)
profitability_result = profitability_chain.invoke(input_data)
stability_result = stability_chain.invoke(input_data)
productivity_result = productivity_chain.invoke(input_data)
liquidity_result = liquidity_chain.invoke(input_data)

# 결과 출력
print("성장성 분석 결과:\n", growth_result.content)
print("수익성 분석 결과:\n", profitability_result)
print("안정성 분석 결과:\n", stability_result)
print("생산성 분석 결과:\n", productivity_result)
print("유동성 분석 결과:\n", liquidity_result)

성장성 분석 결과:
 ### 분석 체인

1. **최신 월(2024-09) 기준 전년 동기(2023-09) 대비 변화율 해석**
   - 2024년 9월 매출 성장률은 6.72%로, 전년 동기인 2023년 9월의 5.22% 대비 28.8% 증가했습니다.
   - 이는 전년 동기 대비 긍정적인 성장을 나타내며, 매출 증가가 지속되고 있음을 시사합니다.
   - 그러나 3년 평균 대비 변화율은 -32.7%로, 장기적인 성장률은 감소 추세에 있습니다.

2. **최근 12개월 동안의 월별 데이터 분석: 계절적 패턴과 월별 추이**
   - 월별 매출 성장률은 2023년 11월과 2024년 1월, 2월에 급격히 상승했으며, 이는 연말 및 연초의 계절적 요인으로 보입니다.
   - 2024년 4월과 5월에는 성장률이 상대적으로 낮아졌으나, 7월과 8월에 다시 상승하는 패턴을 보였습니다.
   - 전반적으로 매출 성장률은 계절적 요인에 따라 변동이 크며, 특정 월에 급격한 상승과 하락이 반복되고 있습니다.

### 경영 전략 체인

1. **계절적 패턴을 활용한 마케팅 전략**
   - 연말과 연초에 매출이 급증하는 경향을 활용하여, 이 시기에 맞춘 집중적인 마케팅 캠페인을 계획하세요.
   - 할인 행사, 프로모션, 신규 제품 출시 등을 통해 매출 상승을 극대화할 수 있습니다.

2. **저성장 월에 대한 대응 전략**
   - 매출 성장률이 낮은 4월과 5월에는 고객 유입을 늘리기 위한 특별 이벤트나 프로모션을 진행하세요.
   - 고객 충성도를 높이기 위한 멤버십 프로그램이나 로열티 프로그램을 강화하여, 비수기에도 안정적인 매출을 유지할 수 있습니다.

3. **장기적인 성장률 개선 방안**
   - 3년 평균 대비 성장률이 감소하고 있으므로, 장기적인 성장 전략을 재검토하세요.
   - 신시장 개척, 제품 다각화, 기술 혁신 등을 통해 지속 가능한 성장을 도모할 필요가 있습니다.
